<a href="https://colab.research.google.com/gist/birdalugureren/084c9e81b950340ed9c4ef77f6bad96c/jstdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Gerekli Paketler

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as offline
import os
from plotly.tools import make_subplots

# Veri Okuma


In [ ]:
if 'output' not in os.listdir():
    os.mkdir('output')

data = pd.read_excel('JSTdatasetR4.xlsx', sheet_name='Data')

all_countries = data['iso'].unique().tolist()

data = data.set_index(['year', 'iso'])

data = data[['imports', 'exports', 'gdp', 'xrusd']]

data = data.dropna()

Veriden bazı sütunları seçtik ve NaN içeren satırları kaldırdık. Verimiz aşağıdaki gibi

In [ ]:
data

,,imports,exports,gdp,xrusd
year,iso,,,,
1870,AUS,36.000,37.000,208.78,0.366946
1871,AUS,34.000,46.000,211.56,0.369146
1872,AUS,38.000,53.000,227.40,0.369239
1873,AUS,49.000,50.000,266.54,0.362405
1874,AUS,49.000,54.000,287.58,0.372223
...,...,...,...,...,...
2012,USA,2763.825,2198.200,16155.30,1.000000
2013,USA,2768.600,2276.625,16691.50,1.000000
2014,USA,2884.075,2375.300,17427.60,1.000000


# Hesaplamalar

Öncelikle aşağıdaki hesaplamaları yapıyoruz:

> $data_{ieg}=
\frac{(imports + exports)}{gdp}
$

>$data_{iex}= 
(imports + exports) \times xrusd
$

>$data_{gx}=
gdp \times xrusd
$

In [ ]:
data_ieg = (data['imports'] + data['exports']) / data['gdp']
data_iex = (data['imports'] + data['exports']) * data['xrusd']
data_gx = data['gdp'] * data['xrusd']

data_ieg = data_ieg.swaplevel()
data_iex = data_iex.swaplevel()
data_gx = data_gx.swaplevel()

### Bütün Ülkelerin Toplamı Alınıyor ve Rate Hesaplanıyor

In [ ]:
total_iex = data_iex.groupby(level='year').sum()
total_gx = data_gx.groupby(level='year').sum()

rate = total_iex / total_gx
rate = rate.sort_index()



# Grafik Oluşturma

In [ ]:

def fill_scatter(d):
    sclist = []
    for country in all_countries:
        _data = d[country].sort_index()
        sclist.append(go.Scatter(x=_data.index, y=_data.values, name=country))
    return sclist


def get_layout(title, yaxis):
    _layout = {
        'title': title,
        'xaxis_title': 'Year',
        'yaxis_title': yaxis
    }
    return _layout



fig_ieg = go.Figure(layout=get_layout('(imports+exports)/gdp', 'rate'))
fig_ieg.add_traces(fill_scatter(data_ieg))

fig_rate = go.Figure(layout=get_layout('rate of iex/gx', 'rate'))
fig_rate.add_trace(go.Scatter(x=rate.index, y=rate.values))

### Oluşturulan Grafikleri Çizme

In [ ]:
fig_ieg.show()
fig_rate.show()